In [2]:

import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=18)
plt.rc('text', usetex=True)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import copy as cp
from typing import Tuple
from sklearn.metrics import confusion_matrix
import json
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline   
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import chi2
import os
import auxiliary as aux_func
from pathlib import Path
import math
import pickle
import lightgbm as lgb
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
MODEL_TYPE='weak'
# MODEL_TYPE='ct3d'
# MODEL_TYPE='ct2d'
# MODEL_TYPE='spm12'
# MODEL_TYPE='spm12contour'
CAL_TYPE = 'slice'
# CAL_TYPE = 'head'
import glob
import shap
FOLDS=5

import re
plt.style.use('classic')
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "stixsans"
plt.rc('figure',facecolor='w')
plt.rc('savefig',dpi=200)


def plot_confusion_matrix(actual_classes : np.array, predicted_classes : np.array, sorted_labels : list):

    matrix = confusion_matrix(actual_classes, predicted_classes, labels=sorted_labels)
    
    plt.figure(figsize=(12.8,6))
    sns.heatmap(matrix, annot=True, xticklabels=sorted_labels, yticklabels=sorted_labels, cmap="Blues", fmt="g")
    plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.title('Confusion Matrix')

    plt.show()



# Per slice

In [3]:
SEARCH_PATH = '/media/akara_bkp/nph_kgreat/kgreathoo/dataset_norm_mask_by_revisited'
nph_files = glob.glob(SEARCH_PATH + '/normal/*/s/*', recursive=True)
NPH = []
for i in nph_files:
    NPH.append(i.split('/')[-1])

### Data exploration

In [4]:
if MODEL_TYPE == 'weak':
    file1 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_code_aj_all_whole.csv'
    file2 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_codeWeak_fixV2.csv'

elif MODEL_TYPE == 'ct3d':
    file1 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_unet3d_fix_per.csv'
    file2 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_unet3d_global.csv'

elif MODEL_TYPE == 'ct2d':
    file1 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_unet2d_fix_per.csv'
    file2 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_unet2d_global.csv'

elif MODEL_TYPE == 'spm12':
    file1 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_spm12_fix_per.csv'
    file2 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_spm12_fix.csv'

elif MODEL_TYPE == 'spm12contour':
    file1 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_spm12contour_fix_per.csv'
    file2 = '/media/akara_bkp/nph_kgreat/kgreathoo/csv/perslice_all_10_spm12contour_fix.csv'

else:
    # Handle an unknown MODEL_TYPE
    raise ValueError(f'Unknown MODEL_TYPE: {MODEL_TYPE}')

#exclude0

# file1 = './csv/perslice_all_10_code_aj_all_whole.csv'
# file2 = './csv/perslice_all_10_codeWeak_exclude0.csv'


# file1 = './csv/perslice_all_10_ct3d.csv'
# file2 = './csv/perslice_all_10_unet3d_exclude0.csv'


# file1 = './csv/perslice_all_10_ct2d.csv'
# file2 = './csv/perslice_all_10_unet2d_exclude0.csv'


# file1 = './csv/perslice_all_10_spm12.csv'
# file2 = './csv/perslice_all_10_spm12_exclude0.csv'


# file1 = './csv/perslice_all_10_spm12contour.csv'
# file2 = './csv/perslice_all_10_spm12contour_exclude0.csv'


# Read CSV files into DataFrames
df1 = pd.read_csv(file1, header=0)
df2 = pd.read_csv(file2, header=0)

df1['y'] = np.where((df1['PatientID']).isin(NPH), 0, 1)
df2['y'] = np.where((df2['PatientID']).isin(NPH), 0, 1)

# Check the condition
if CAL_TYPE != 'head':
    # Extract columns from the first CSV file
    cols = []
    cols2 = []
    for i in range(10):
        cols.append(f'CSF ratio_{i}')
        cols.append(f'CSF / W+G matter ratio_{i}')
        cols.append(f"Mean CSF ratio_{i}")
        cols.append(f"Min CSF ratio_{i}")
        cols.append(f"Max CSF ratio_{i}")
        cols.append(f"Std CSF ratio_{i}")
        cols.append(f"Mean CSF / W+G matter ratio_{i}")
        cols.append(f"Min CSF / W+G matter ratio_{i}")
        cols.append(f"Max CSF / W+G matter ratio_{i}")
        cols.append(f"Std CSF / W+G matter ratio_{i}")

    # Extract additional columns from the first CSV file
    cols2.append('CSF ratio_all')
    cols2.append('CSF / W+G matter ratio_all')

    # Create new DataFrames with selected columns
    df2_selected = df2[cols2]
    df1[cols2] = df2_selected

data = df1
data['HN']=data['HN'].astype(str)

if CAL_TYPE!='head':
    new_data = data.fillna(0)
    data = new_data.groupby(['HN', 'fold'], sort=True).max(numeric_only=True).reset_index()

mismatch = pd.read_csv('p_pun_217.csv')
mismatch['HN'] = mismatch['HN'].astype(str)
merged_df = pd.merge(data[['HN', 'y']], mismatch[['HN', 'y']], on='HN', suffixes=('_data', '_mismatch'))
diff_df = merged_df[merged_df['y_data'] != merged_df['y_mismatch']]
hn_list = diff_df['HN'].tolist()
mask = data['HN'].isin(hn_list)
data.loc[mask, 'y'] = 0

In [5]:
'''
_sum_all is old method that sum of each partition -> not correct
'''
if 'fix' not in file1 and 'fix' not in file2 and 'exclude' not in file1 and 'exclude' not in file2 and 'global' not in file1 and 'global' not in file2:
    print('old')
    data['CSF_ratio_sum_all'] = data[[f'CSF ratio_{i}' for i in range(10)]].sum(axis=1)/10
    data['CSF / W+G matter ratio_sum_all'] = data[[f'CSF / W+G matter ratio_{i}' for i in range(10)]].sum(axis=1)/10
else:
    data['CSF_ratio_all'] = data['CSF ratio_all']
    data['CSF_ratio_sum_all'] = data[[f'CSF ratio_{i}' for i in range(10)]].sum(axis=1)/10
    data['CSF / W+G matter ratio_sum_all'] = data[[f'CSF / W+G matter ratio_{i}' for i in range(10)]].sum(axis=1)/10


In [6]:
# '''
# reindex from 0-9 to 1-10

# This code block reindexes the column names of the 'data' DataFrame. It replaces the numbers in the column names with incremented numbers, starting from 1. It also adds additional columns based on the original column names.
# '''
# if MODEL_TYPE == 'weak':
#     data.columns = [
#     re.sub(r'(\w+) CSF ratio_(\d+)', lambda match: f'{match.group(1)} CSF ratio_{int(match.group(2)) + 1}', col)
#     for col in data.columns 
#     ]
#     data.columns  = [
#     re.sub(r'^CSF ratio_(\d+)', lambda match: f'CSF ratio_{int(match.group(1)) + 1}', col)
#     for col in data.columns 
#     ]

#     if CAL_TYPE!='head':
#         cols=[]
#         for i in range(1,11):
#             cols.append(f'CSF ratio_{i}')     
#             cols.append(f'CSF / W+G matter ratio_{i}')
#             cols.append(f"Mean CSF ratio_{i}")
#             cols.append(f"Min CSF ratio_{i}")
#             cols.append(f"Max CSF ratio_{i}")
#             cols.append(f"Std CSF ratio_{i}")
#             cols.append(f"Mean CSF / W+G matter ratio_{i}")
#             cols.append(f"Min CSF / W+G matter ratio_{i}")
#             cols.append(f"Max CSF / W+G matter ratio_{i}")
#             cols.append(f"Std CSF / W+G matter ratio_{i}")
#         cols.append('CSF_ratio_all')
#         cols.append('CSF / W+G matter ratio_all')
#         cols.append('CSF_ratio_sum_all')
#         cols.append('CSF / W+G matter ratio_sum_all')


### Implementing the model

In [7]:

train_test_folds = json.load(open('folds5_42_val.json'))

folds={}
num_fold=FOLDS
for fold_idx in range(num_fold):
    folds[fold_idx] = {
        'train':[i.split('/')[-1].split('-')[0].split('s')[1][0:10] for i in train_test_folds[str(fold_idx)]['train']],
        'test':[i.split('/')[-1].split('-')[0].split('s')[1][0:10]  for i in train_test_folds[str(fold_idx)]['test']],
        'val':[i.split('/')[-1].split('-')[0].split('s')[1][0:10]  for i in train_test_folds[str(fold_idx)]['val']],
    }
    fold_idx+=1

data['type'] = None

for fold_idx in range(num_fold):
    data.loc[(data['fold'] == int(fold_idx)) & data['HN'].isin(folds[fold_idx]['train']), 'type'] = f'train{fold_idx}'
    data.loc[(data['fold'] == int(fold_idx)) & data['HN'].isin(folds[fold_idx]['test']), 'type'] = f'test{fold_idx}'

data.drop_duplicates(subset=['HN','fold'], inplace=True)

In [8]:


"""
This code snippet defines a list of column names based on the value of the MODEL_TYPE variable that from previose features selection.
The column names are modified using regular expressions to increment the CSF ratio values by 1.
"""

if MODEL_TYPE == 'weak':
    cols = ['CSF ratio_5', 'Std CSF ratio_8', 'Mean CSF ratio_5',
            'Max CSF ratio_8', 'Max CSF ratio_6', 'Min CSF ratio_5',
            'Mean CSF ratio_8', 'Min CSF ratio_4', 'Max CSF ratio_7',
            'CSF ratio_4']

elif MODEL_TYPE == 'ct3d':
    cols =  ['CSF ratio_0',
     'Mean CSF ratio_0',
     'CSF / W+G matter ratio_0',
     'Max CSF ratio_0',
     'Mean CSF / W+G matter ratio_0',
     'CSF ratio_9',
     'Std CSF / W+G matter ratio_9',
     'Max CSF / W+G matter ratio_9',
     'Min CSF / W+G matter ratio_9',
     'Mean CSF / W+G matter ratio_9',
     'Std CSF ratio_9',
     'Max CSF ratio_9',
     'Min CSF ratio_9',
     'Mean CSF ratio_9',
     'CSF / W+G matter ratio_9',
     'Min CSF / W+G matter ratio_8',
     'Mean CSF ratio_5',
     'Std CSF / W+G matter ratio_8',
     'Mean CSF / W+G matter ratio_7',
     'Mean CSF / W+G matter ratio_8',
     'Std CSF ratio_8',
     'Max CSF ratio_8',
     'Min CSF ratio_8',
     'Mean CSF ratio_8',
     'CSF / W+G matter ratio_8',
     'CSF ratio_8',
     'Min CSF / W+G matter ratio_7',
     'Max CSF ratio_7',
     'Std CSF ratio_7',
     'Max CSF / W+G matter ratio_7',
     'Min CSF ratio_7',
     'Mean CSF ratio_7',
     'CSF / W+G matter ratio_7',
     'CSF ratio_7',
     'Std CSF / W+G matter ratio_6',
     'Max CSF / W+G matter ratio_6',
     'Min CSF / W+G matter ratio_6',
     'Mean CSF / W+G matter ratio_6',
     'Std CSF ratio_6',
     'Max CSF ratio_6',
     'Min CSF ratio_6',
     'Mean CSF ratio_6',
     'CSF / W+G matter ratio_6',
     'CSF ratio_6',
     'Std CSF / W+G matter ratio_5',
     'Max CSF / W+G matter ratio_5',
     'Min CSF / W+G matter ratio_5',
     'Mean CSF / W+G matter ratio_5',
     'Std CSF ratio_5',
     'Max CSF ratio_5',
     'Min CSF ratio_5',
     'Std CSF / W+G matter ratio_7',
     'Std CSF / W+G matter ratio_4',
     'Max CSF / W+G matter ratio_8',
     'CSF / W+G matter ratio_5',
     'Max CSF / W+G matter ratio_4',
     'Min CSF / W+G matter ratio_4',
     'Mean CSF / W+G matter ratio_4',
     'Std CSF ratio_4',
     'Max CSF ratio_4',
     'Min CSF ratio_4',
     'Mean CSF ratio_4',
     'Min CSF / W+G matter ratio_2',
     'CSF ratio_4',
     'Std CSF / W+G matter ratio_3',
     'CSF / W+G matter ratio_4',
     'Min CSF / W+G matter ratio_3',
     'Mean CSF / W+G matter ratio_3',
     'Std CSF ratio_3',
     'Max CSF ratio_3',
     'Min CSF ratio_3',
     'Mean CSF ratio_3',
     'CSF / W+G matter ratio_3',
     'CSF ratio_3',
     'Std CSF / W+G matter ratio_2',
     'Max CSF / W+G matter ratio_3',
     'Max CSF / W+G matter ratio_2',
     'Max CSF ratio_2',
     'Mean CSF / W+G matter ratio_2',
     'Std CSF ratio_2',
     'CSF_ratio_all',
     'Min CSF ratio_2',
     'Mean CSF ratio_2',
     'CSF / W+G matter ratio_2',
     'CSF ratio_2',
     'Std CSF / W+G matter ratio_1',
     'Max CSF / W+G matter ratio_1',
     'Min CSF / W+G matter ratio_1',
     'Mean CSF / W+G matter ratio_1',
     'Std CSF ratio_1',
     'Max CSF ratio_1',
     'Min CSF ratio_1',
     'Mean CSF ratio_1',
     'CSF / W+G matter ratio_1',
     'CSF ratio_1',
     'Std CSF / W+G matter ratio_0',
     'Max CSF / W+G matter ratio_0',
     'Min CSF / W+G matter ratio_0',
     'Std CSF ratio_0',
     'Min CSF ratio_0']

elif MODEL_TYPE == 'ct2d':
    cols = ['CSF / W+G matter ratio_all', 'CSF_ratio_all', 'Max CSF ratio_4',
       'CSF ratio_5', 'Mean CSF / W+G matter ratio_5', 'Max CSF ratio_5',
       'Mean CSF / W+G matter ratio_4', 'Max CSF / W+G matter ratio_5',
       'Min CSF ratio_4', 'Mean CSF ratio_6',
       'Min CSF / W+G matter ratio_4', 'Std CSF / W+G matter ratio_8',
       'Max CSF / W+G matter ratio_3', 'Min CSF ratio_6',
       'Min CSF / W+G matter ratio_6', 'Max CSF / W+G matter ratio_6',
       'CSF ratio_4', 'Std CSF ratio_8', 'Std CSF / W+G matter ratio_4',
       'Mean CSF / W+G matter ratio_6']

elif MODEL_TYPE == 'spm12':
    cols = ['Mean CSF ratio_5', 'Min CSF ratio_2', 'Mean CSF ratio_4',
       'Mean CSF / W+G matter ratio_5', 'Mean CSF ratio_7', 'CSF ratio_9',
       'Max CSF / W+G matter ratio_5', 'Min CSF / W+G matter ratio_5',
       'CSF ratio_2', 'CSF ratio_8', 'Min CSF / W+G matter ratio_4',
       'Mean CSF / W+G matter ratio_7', 'Max CSF ratio_7',
       'Max CSF ratio_5', 'CSF ratio_1', 'Max CSF ratio_3', 'CSF ratio_3',
       'CSF ratio_5', 'Max CSF ratio_8', 'CSF / W+G matter ratio_7',
       'Min CSF / W+G matter ratio_7', 'Max CSF / W+G matter ratio_4',
       'Min CSF ratio_5', 'CSF ratio_4', 'Min CSF / W+G matter ratio_2',
       'Max CSF ratio_4', 'CSF / W+G matter ratio_2',
       'CSF / W+G matter ratio_4', 'Max CSF / W+G matter ratio_8',
       'Std CSF ratio_1', 'CSF / W+G matter ratio_3',
       'Std CSF / W+G matter ratio_7', 'Mean CSF / W+G matter ratio_3',
       'Std CSF / W+G matter ratio_3', 'Mean CSF ratio_2', 'CSF ratio_7',
       'Std CSF / W+G matter ratio_6', 'Min CSF ratio_7',
       'Mean CSF / W+G matter ratio_4', 'Std CSF / W+G matter ratio_5',
       'Min CSF ratio_8', 'Std CSF ratio_3',
       'Min CSF / W+G matter ratio_6', 'Min CSF ratio_4',
       'Std CSF / W+G matter ratio_4', 'Max CSF / W+G matter ratio_3',
       'CSF_ratio_sum_all', 'CSF / W+G matter ratio_1', 'Min CSF ratio_3',
       'Std CSF ratio_8', 'Std CSF ratio_4',
       'Min CSF / W+G matter ratio_1', 'Min CSF / W+G matter ratio_8',
       'Std CSF ratio_7', 'Max CSF / W+G matter ratio_1',
       'CSF / W+G matter ratio_5', 'Mean CSF ratio_6', 'Std CSF ratio_2',
       'CSF / W+G matter ratio_6', 'CSF / W+G matter ratio_sum_all',
       'CSF_ratio_all', 'Min CSF ratio_6', 'Std CSF ratio_6',
       'Mean CSF / W+G matter ratio_2', 'Max CSF ratio_2',
       'Max CSF / W+G matter ratio_2', 'Mean CSF ratio_3',
       'Std CSF / W+G matter ratio_1', 'Mean CSF / W+G matter ratio_6',
       'Min CSF / W+G matter ratio_3', 'Max CSF / W+G matter ratio_7',
       'Mean CSF / W+G matter ratio_1', 'Std CSF ratio_5',
       'Max CSF / W+G matter ratio_6', 'Max CSF ratio_6',
       'CSF / W+G matter ratio_8', 'Max CSF ratio_1',
       'Std CSF / W+G matter ratio_8', 'Min CSF ratio_1',
       'Mean CSF / W+G matter ratio_8', 'Mean CSF ratio_1',
       'Mean CSF ratio_8', 'Std CSF / W+G matter ratio_2', 'CSF ratio_6',
       'CSF / W+G matter ratio_all', 'Std CSF ratio_9',
       'Max CSF / W+G matter ratio_9', 'Mean CSF ratio_9',
       'Max CSF ratio_9', 'Mean CSF / W+G matter ratio_9',
       'Std CSF / W+G matter ratio_9', 'Min CSF ratio_9',
       'Min CSF / W+G matter ratio_9', 'CSF / W+G matter ratio_9']

elif MODEL_TYPE == 'spm12contour':
    cols = ['Min CSF ratio_7', 'Std CSF ratio_3', 'Mean CSF ratio_8',
       'Std CSF / W+G matter ratio_3', 'Max CSF / W+G matter ratio_8',
       'Mean CSF / W+G matter ratio_4', 'CSF ratio_4',
       'Min CSF / W+G matter ratio_5', 'Min CSF ratio_4',
       'CSF / W+G matter ratio_7', 'CSF / W+G matter ratio_8',
       'Min CSF / W+G matter ratio_8', 'Max CSF ratio_8',
       'Mean CSF / W+G matter ratio_5', 'Std CSF ratio_1',
       'CSF_ratio_sum_all', 'Max CSF ratio_4',
       'Mean CSF / W+G matter ratio_8', 'Mean CSF ratio_5', 'CSF ratio_8',
       'Std CSF / W+G matter ratio_1', 'Min CSF / W+G matter ratio_4',
       'Max CSF / W+G matter ratio_6', 'Mean CSF ratio_2',
       'Mean CSF ratio_4', 'Min CSF ratio_5', 'CSF_ratio_all',
       'CSF / W+G matter ratio_2', 'CSF / W+G matter ratio_4',
       'Std CSF / W+G matter ratio_5', 'Min CSF ratio_2',
       'Min CSF / W+G matter ratio_7', 'Max CSF ratio_6',
       'Max CSF / W+G matter ratio_3', 'Mean CSF ratio_7', 'CSF ratio_5',
       'CSF ratio_1', 'Max CSF ratio_3', 'Max CSF / W+G matter ratio_5',
       'CSF ratio_2', 'Min CSF ratio_3', 'Min CSF / W+G matter ratio_2',
       'Std CSF ratio_8', 'Mean CSF ratio_3', 'Min CSF ratio_8',
       'CSF ratio_7', 'Std CSF ratio_5', 'CSF / W+G matter ratio_sum_all',
       'Max CSF ratio_1', 'Std CSF / W+G matter ratio_6',
       'Min CSF / W+G matter ratio_1', 'CSF ratio_3',
       'Min CSF / W+G matter ratio_3', 'Max CSF / W+G matter ratio_2',
       'Max CSF ratio_5', 'Std CSF / W+G matter ratio_8',
       'CSF / W+G matter ratio_3', 'Mean CSF / W+G matter ratio_1',
       'Max CSF / W+G matter ratio_4', 'CSF / W+G matter ratio_5',
       'Std CSF ratio_6', 'Std CSF ratio_7', 'Mean CSF ratio_6',
       'Min CSF ratio_1', 'Mean CSF / W+G matter ratio_7',
       'Std CSF / W+G matter ratio_4', 'Std CSF / W+G matter ratio_2',
       'Max CSF ratio_2', 'Max CSF ratio_7',
       'Mean CSF / W+G matter ratio_3', 'Max CSF / W+G matter ratio_7',
       'Max CSF / W+G matter ratio_1', 'CSF ratio_6',
       'CSF / W+G matter ratio_all', 'Std CSF ratio_4',
       'CSF / W+G matter ratio_6', 'Mean CSF / W+G matter ratio_9',
       'Mean CSF ratio_1', 'Mean CSF ratio_9',
       'Mean CSF / W+G matter ratio_2', 'Max CSF / W+G matter ratio_9',
       'Mean CSF / W+G matter ratio_6', 'Std CSF / W+G matter ratio_9',
       'Min CSF / W+G matter ratio_6', 'Min CSF ratio_6',
       'Std CSF ratio_9', 'CSF / W+G matter ratio_1', 'Std CSF ratio_2',
       'Max CSF ratio_9', 'CSF ratio_9', 'Std CSF / W+G matter ratio_7',
       'CSF / W+G matter ratio_9', 'Min CSF / W+G matter ratio_9',
       'Min CSF ratio_9']

else:
    # Handle an unknown MODEL_TYPE
    raise ValueError(f'Unknown MODEL_TYPE: {MODEL_TYPE}')

if CAL_TYPE=='head':
    cols = ["CSF ratio_all","CSF / W+G matter ratio_sum_all"]


In [ ]:
# model='lr'
# model='RF'
model='light'
# model='load'
y_true = []
y_pred = []
y_prob = []
X=data[cols].to_numpy()
y=data['y']
Y=y.astype('int')
score_train=[]
score_test=[]
xx, yy = np.mgrid[-1:1.1:.01, -1:1.1:.01]
grid = np.c_[xx.ravel(), yy.ravel()]
save_idx=[]
pred_df = pd.DataFrame()
for fold_idx in range(FOLDS):
    pred_df_tmp = pd.DataFrame()
    if '42' in file1 or 'code' in file1 :
        print('codeeee : ')
        data['type'] = None
        data['type'] = np.where((data['fold'] == int(fold_idx)) & data['HN'].isin(folds[fold_idx]['train']), f'train{fold_idx}', data['type'])
        data['type'] = np.where((data['fold'] == int(fold_idx)) & data['HN'].isin(folds[fold_idx]['test']), f'test{fold_idx}', data['type'])

    else:
        data['type']=None
        data['type'] = np.where(data['HN'].isin(folds[fold_idx]['train']), f'train{fold_idx}', data['type'])
        data['type'] = np.where(data['HN'].isin(folds[fold_idx]['test']), f'test{fold_idx}',data['type'])
        data['type'] = np.where(data['HN'].isin(folds[fold_idx]['val']), f'val{fold_idx}', data['type'])
        
        data['type'] = data.apply(lambda row: None if (row['type'] == f'test{fold_idx}' and row['HN'] in folds[fold_idx]['train']) else row['type'], axis=1)

    X_train = data[(data['type']==f'train{fold_idx}') | (data['type']==f'val{fold_idx}')][cols].to_numpy()
    y_train = data[(data['type']==f'train{fold_idx}') | (data['type']==f'val{fold_idx}')]['y'].to_numpy()
    X_test = data[data['type']==f'test{fold_idx}'][cols].to_numpy()
    y_test = data[data['type']==f'test{fold_idx}']['y'].to_numpy()
    test = train_test_folds['{}'.format(fold_idx)]['test']
    save_idx.append(data.index[data['type']==f'test{fold_idx}'].tolist())
    pred_df_tmp['HN'] =data[data['type']==f'test{fold_idx}']['HN']
    pred_df_tmp['y']=data[data['type']==f'test{fold_idx}']['y']


    if(model=='lr'):
        C=100
        model2 = LogisticRegression(class_weight="balanced", C=C,solver='lbfgs' ,max_iter=10000)
        model2 = LogisticRegression(C=C).fit(X_train, y_train)
    elif(model=='RF'):
        model2 = RandomForestClassifier(class_weight="balanced",n_estimators=5)
        model2.fit(X_train, y_train)
    elif(model=='light'):
#         num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, 
        model2 = lgb.LGBMClassifier(force_col_wise=True,class_weight="balanced",num_leaves=2, max_depth=0)
        model2.fit(X_train, y_train)
    else:
        model2 = pickle.load(open(f'/data2-2tb/kgreathoo/Weakconfident1{fold_idx}_89_2025.sav', 'rb'))
    print("%3d\t%.5f\t\t%.5f" % 
          (fold_idx, model2.score(X_train, y_train), model2.score(X_test, y_test)))

#     features_name = data[cols]
    

#     features_name.columns = [re.sub(r'^CSF ratio_(\d+)', r'$\omega^{(\1)}$', col) for col in features_name.columns]
#     features_name.columns = [re.sub(r'(\w+) CSF ratio_(\d+)', r'\1 $(\omega)^{(\2)}$', col) for col in features_name.columns]
#     features_name.columns = [re.sub(r'Std' , r'Std.', col) for col in features_name.columns]

#     if fold_idx==0:
#         explainer = shap.TreeExplainer(model2, X_train, feature_perturbation="interventional")
#         shap_values = explainer(features_name)
#         fig = plt.figure(figsize=(30, 28), dpi=300)
#         fig = shap.plots.beeswarm(shap_values, max_display=10, show=False)
# #         plt.title(title_map[MODEL_TYPE], fontsize=18, fontweight='bold')
# #         plt.savefig(f'SHAP10Weak.pdf', bbox_inches='tight')
#         plt.show()
    score_train.append(model2.score(X_train, y_train))
    score_test.append(model2.score(X_test, y_test))
    
    y_true.extend(y_test)
    y_pred.extend(model2.predict(X_test))
    y_prob.extend(model2.predict_proba(X_test))
    pickle.dump(model2, open(f'Weakconfident1{fold_idx}_89_2025.sav', 'wb'))
#     pickle.dump(model2, open(f'SPM12confident1contour{fold_idx}.sav', 'wb'))
#     pickle.dump(model2, open(f'SPM12confident1{fold_idx}.sav', 'wb'))
#     pickle.dump(model2, open(f'headCT2023v2{fold_idx}.sav', 'wb'))
#     pickle.dump(model2, open(f'ct3d{fold_idx}.sav', 'wb'))

    pred_df = pd.concat([pred_df, pred_df_tmp], ignore_index=True)
pred_df['y_pred_model']=y_pred
pred_df['y_prob']=y_prob


# if(model=='lr'):
#     explainer = shap.LinearExplainer(model2, X_train, feature_perturbation="interventional")
#     shap_values = explainer(data[cols])
#     shap.plots.beeswarm(shap_values, max_display=200)
# elif(model=='RF'):
#     explainer = shap.TreeExplainer(model2)
#     shap_values = explainer(X_test)
#     shap_values = explainer.shap_values(data[cols]) 
#     shap.summary_plot(shap_values[1], data[cols]) 
# elif(model=='light'):
#     explainer = shap.TreeExplainer(model2, X_train, feature_perturbation="interventional")
#     shap_values = explainer(data[cols])
#     shap.plots.beeswarm(shap_values, max_display=200)

print(file1,'Accuracy: %.3f (%.3f)' % (mean(score_test), std(score_test)))
y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred)
y_prob = np.asarray(y_prob)
print
confusion_matrix(y_true, y_pred, labels=[0, 1])
logit_roc_auc = roc_auc_score(y_true, y_prob[:, 1])
fpr, tpr, thresholds = roc_curve(y_true, y_prob[:,1])
print(classification_report(y_true, y_pred), f' (area = %0.2f)' % logit_roc_auc)
from sklearn.metrics import cohen_kappa_score
cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
TP, FN = cm[0]
FP, TN = cm[1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
f1_score = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
roc_auc = roc_auc_score(y_true, y_prob[:, 1])
cohen_kappa = cohen_kappa_score(y_true, y_pred)

# print("Acc\tSensiy\tSpec\tF1-Score\tTP\tTN\tFP\tFN\tAU-ROC\tCohen")
print(f"{accuracy:.3f}\t{sensitivity:.3f}\t{specificity:.3f}\t{f1_score:.3f}\t"
      f"{TP}\t{TN}\t{FP}\t{FN}\t{roc_auc:.3f}\t{cohen_kappa:.3f}")





In [9]:
import pandas as pd
import shap
import pickle
import matplotlib.pyplot as plt

# โหลด interest case ที่สนใจ (เฉพาะ test fold)
abnormal_info = pd.read_csv("/data4-4tb/kgreathoo/abnormal_cases_info.csv")
abnormal_info['HN'] = abnormal_info['HN'].astype(int)
abnormal_info = abnormal_info[abnormal_info['type'].str.contains('test')]
abnormal_info['fold'] = abnormal_info['type'].str.extract(r'test(\d+)').astype(int)

import os
import pickle
import shap
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob

def find_partition_img(hn: int, plots_dir='/data4-4tb/kgreathoo/plots_result'):
    # ค้นหาไฟล์ที่มี HN อยู่ในชื่อ
    matching_files = glob.glob(os.path.join(plots_dir, f"*{hn}*.png"))
    print(matching_files)
    if matching_files:
        return matching_files[0],matching_files[0].split('/')[-1]  # เอาไฟล์แรกที่เจอ
    return None

def generate_pretty_title(filename):
    base = os.path.basename(filename)
    base = base.replace('_', ' ').replace('.png', '')
    # ลบ "peak idx" เป็น "Peak" และ "type" เป็น "Type"
    base = base.replace('peak idx', 'Peak Index')
    base = base.replace('type', 'Type')
    base = base.replace('HN', 'HN')
    # เติมความเป็นระเบียบ
    return base.title()

output_dir = "shap_outputs_1_10_new"
os.makedirs(output_dir, exist_ok=True)

for i, row in abnormal_info.iterrows():
    hn = row['HN']
    fold = int(row['fold'])
    peak = int(row['peak_partition'])+1
    type_name = row['type']

    # โหลดโมเดล
#     model_path = f'/data2-2tb/kgreathoo/Weakconfident1{fold}_89.sav'
    model_path = f'/media/akara_bkp/nph_kgreat/kgreathoo/Weakconfident1{fold}_89.sav'
    if not os.path.exists(model_path):
        print(f"❌ ไม่พบโมเดล: {model_path}")
        continue
    model = pickle.load(open(model_path, 'rb'))

    # เตรียมข้อมูล
    person_row = data[(data['HN'].astype(int) == hn) & (data['fold'] == fold)]
    if person_row.empty:
        print(f"❌ ไม่พบ HN={hn}, fold={fold}")
        continue
    X_sample = person_row[cols].values.reshape(1, -1)

    # คำนวณ SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)

    # บันทึก SHAP เป็นไฟล์เดี่ยว
    shap_img_path = os.path.join(output_dir, f'shap_HN{hn}_fold{fold}.png')
    plt.figure()
    
    def adjust_partition_index(name):
        return re.sub(r'_(\d+)', lambda m: f'_{int(m.group(1)) + 1}', name)

    adjusted_cols = [adjust_partition_index(col) for col in cols]
    shap.plots.waterfall(
        shap.Explanation(
            values=shap_values[1][0],
            base_values=explainer.expected_value[1],
            data=X_sample[0],
            feature_names=adjusted_cols
        ),
        show=False
    )

    plt.tight_layout()
    plt.savefig(shap_img_path, dpi=150, bbox_inches='tight')
    plt.close()

    # เตรียม path ของภาพ partition
    img_path, img_name = find_partition_img(hn)
    if img_path:
        title = generate_pretty_title(img_path)
        print(f"✅ Image: {img_path}")
        print(f"🖼️ Title: {title}")
    else:
        print(f"❌ ไม่พบภาพของ HN: {hn}")
    if not os.path.exists(img_path):
        print(f"❌ ไม่พบภาพ CT: {img_path}")
        continue

    # รวมภาพ SHAP + CT partition
    shap_img = Image.open(shap_img_path)
    partition_img = Image.open(img_path)

    # ปรับขนาดให้เท่ากันด้านสูง
    h = max(shap_img.height, partition_img.height)
#     print(h)
    partition_img = partition_img.resize((int(shap_img.width * h / shap_img.height), h))


    combined = Image.new("RGB", (shap_img.width + partition_img.width, h), color=(255, 255, 255))
    combined.paste(shap_img, (0, 0))
    combined.paste(partition_img, (shap_img.width, 0))

    final_path = os.path.join(output_dir, img_name)
    combined.save(final_path)
    print(f"✅ Saved combined image: {final_path}")

    # ลบภาพเดี่ยว (ถ้าไม่ต้องการเก็บไว้)
    os.remove(shap_img_path)
    break


Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


['/data4-4tb/kgreathoo/plots_result/38160661_peak_idx_1_type_test3_TP.png']
✅ Image: /data4-4tb/kgreathoo/plots_result/38160661_peak_idx_1_type_test3_TP.png
🖼️ Title: 38160661 Peak Index 1 Type Test3 Tp
✅ Saved combined image: shap_outputs_1_10_new/38160661_peak_idx_1_type_test3_TP.png


In [ ]:
pred_df['HN'] = pred_df['HN'].astype(str)

In [ ]:
# pred_df.to_csv('pred_weak_89.csv')

In [ ]:
pred_df['proba_score'] = [prob[1] for prob in y_prob]

# Aj.Akara

In [43]:
import pandas as pd
import shap
import pickle
import matplotlib.pyplot as plt

# โหลด interest case ที่สนใจ (เฉพาะ test fold)
abnormal_info = pd.read_csv("/data4-4tb/kgreathoo/abnormal_cases_info.csv")
abnormal_info['HN'] = abnormal_info['HN'].astype(int)
abnormal_info = abnormal_info[abnormal_info['type'].str.contains('test')]
abnormal_info['fold'] = abnormal_info['type'].str.extract(r'test(\d+)').astype(int)

import os
import pickle
import shap
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import re
FONT_SIZE = 96

def find_partition_img(hn: int, plots_dir='/data4-4tb/kgreathoo/plots_result_akara'):
    # ค้นหาไฟล์ที่มี HN อยู่ในชื่อ
    matching_files = glob.glob(os.path.join(plots_dir, f"*{hn}*.png"))
    print(matching_files)
    if matching_files:
        return matching_files[0],matching_files[0].split('/')[-1]  # เอาไฟล์แรกที่เจอ
    return None, None

def generate_pretty_title(filename):
    base = os.path.basename(filename)
    base = base.replace('_', ' ').replace('.png', '')
    # ลบ "peak idx" เป็น "Peak" และ "type" เป็น "Type"
    base = base.replace('peak idx', 'Peak Index')
    base = base.replace('type', 'Type')
    base = base.replace('HN', 'HN')
    # เติมความเป็นระเบียบ
    return base.title()

output_dir = "shap_outputs_1_10_new_great"
os.makedirs(output_dir, exist_ok=True)

for i, row in abnormal_info.iterrows():
    hn = row['HN']
    fold = int(row['fold'])
    peak = int(row['peak_partition'])+1
    type_name = row['type']
    img_path, img_name = find_partition_img(hn)
    if img_path is None:
        print(f'skip: {hn}')
        continue

    # โหลดโมเดล
#     model_path = f'/data2-2tb/kgreathoo/Weakconfident1{fold}_89.sav'
    model_path = f'/media/akara_bkp/nph_kgreat/kgreathoo/Weakconfident1{fold}_89.sav'
    if not os.path.exists(model_path):
        print(f"❌ ไม่พบโมเดล: {model_path}")
        continue
    model = pickle.load(open(model_path, 'rb'))

    # เตรียมข้อมูล
    person_row = data[(data['HN'].astype(int) == hn) & (data['fold'] == fold)]
    if person_row.empty:
        print(f"❌ ไม่พบ HN={hn}, fold={fold}")
        continue
    X_sample = person_row[cols].values.reshape(1, -1)

    # คำนวณ SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)

    # บันทึก SHAP เป็นไฟล์เดี่ยว
    shap_img_path = os.path.join(output_dir, f'shap_HN{hn}_fold{fold}.png')
    plt.figure()
    
    def adjust_partition_index(name):
        return re.sub(r'_(\d+)', lambda m: f'_{int(m.group(1)) + 1}', name)

    adjusted_cols = [adjust_partition_index(col) for col in cols]
    adjusted_cols = [f.replace(' CSF ratio',' ($\omega$)') for f in adjusted_cols]
    adjusted_cols = [f.replace('CSF ratio','$\omega$') for f in adjusted_cols]
    adjusted_cols = [f.replace('Std ','Std. ') for f in adjusted_cols]
    
    def replace_csf_with_omega_sup(name):
        return re.sub(r'_(\d+)', r'$^{(\1)}$', name)
    
    adjusted_cols = [f.replace('CSF ratio','$\omega$') for f in adjusted_cols]
    adjusted_cols = [replace_csf_with_omega_sup(f) for f in adjusted_cols]
    
    print(adjusted_cols)

    plt.rcParams.update({'font.size': FONT_SIZE})
    shap.plots.waterfall(
        shap.Explanation(
            values=shap_values[1][0],
            base_values=explainer.expected_value[1],
            data=X_sample[0],
            feature_names=adjusted_cols
        ),
        show=False
    )

    plt.tight_layout()
    plt.savefig(shap_img_path, dpi=150, bbox_inches='tight')
    plt.close()

    # เตรียม path ของภาพ partition
    img_path, img_name = find_partition_img(hn)
    if img_path:
        title = generate_pretty_title(img_path)
        print(f"✅ Image: {img_path}")
        print(f"🖼️ Title: {title}")
    else:
        print(f"❌ ไม่พบภาพของ HN: {hn}")
    if not os.path.exists(img_path):
        print(f"❌ ไม่พบภาพ CT: {img_path}")
        continue

###############
        
#     # รวมภาพ SHAP + CT partition
#     shap_img = Image.open(shap_img_path)
#     partition_img = Image.open(img_path)
    
#     if partition_img.mode in ("RGBA", "LA"):
#         white_bg = Image.new("RGB", partition_img.size, (255, 255, 255))
#         white_bg.paste(partition_img, mask=partition_img.split()[-1])  # use its own alpha as mask
#         partition_img = white_bg          # now an opaque RGB image

#     # ปรับขนาดให้เท่ากันด้านสูง
#     h = max(shap_img.height, partition_img.height)
# #     print(h)
#     partition_img = partition_img.resize((int(shap_img.width * h / shap_img.height), h))


#     combined = Image.new("RGB", (shap_img.width + partition_img.width, h), color=(255, 255, 255))
#     combined.paste(shap_img, (0, 0))
#     combined.paste(partition_img, (shap_img.width, 0))

#     final_path = os.path.join(output_dir, img_name)
#     combined.save(final_path)
#     print(f"✅ Saved combined image: {final_path}")

#################

    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np
    import os

    def rgba_to_rgb_white(arr: np.ndarray) -> np.ndarray:
        """
        Convert an RGBA image array (H, W, 4) to opaque RGB on white.
        Matplotlib returns floats in [0,1]; the formula works directly.
        """
        if arr.ndim == 3 and arr.shape[-1] == 4:
            alpha = arr[..., 3:4]         # keep axis for broadcasting
            rgb   = arr[..., :3]
            return rgb * alpha + (1 - alpha)   # white background
        return arr

    # 1) read each PNG into arrays
    shap_arr = mpimg.imread(shap_img_path)          # already RGB (no alpha)
    part_arr = mpimg.imread(img_path)               # may be RGBA
    part_arr = rgba_to_rgb_white(part_arr)

    # 2) bring both arrays to the same pixel height while
    #    keeping each aspect ratio (no interpolation artefacts)
    target_h = max(shap_arr.shape[0], part_arr.shape[0])

#     def resize_keep_aspect(img, target_h):
#         """Resize a NumPy HxWxC image to target_h pixels high via matplotlib."""
#         h, w = img.shape[:2]
#         if h == target_h:
#             return img
#         scale = target_h / h
#         fig = plt.figure(figsize=(w * scale / 100, target_h / 100), dpi=100)
#         ax = fig.add_axes([0, 0, 1, 1])
#         ax.axis('off')
#         ax.imshow(img)
#         fig.canvas.draw()
#         resized = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
#         resized = resized.reshape(target_h, int(w * scale), 4)[:, :, [1, 2, 3]] / 255.0
#         plt.close(fig)
#         return resized
    def resize_keep_aspect(img: np.ndarray, target_h: int) -> np.ndarray:
        """
        Resize an HxWxC image to the desired height while
        preserving aspect ratio. Works for uint8 [0-255] or
        float [0-1] arrays, with or without alpha.
        """
        h, w = img.shape[:2]
        if h == target_h:          # nothing to do
            return img

        # Pillow expects uint8.  Convert if necessary
        original_dtype = img.dtype
        if img.dtype != np.uint8:
            img_uint8 = (img * 255).astype(np.uint8)  # assumes 0-1 float
        else:
            img_uint8 = img

        pil_img = Image.fromarray(img_uint8)
        new_w   = int(round(w * target_h / h))

        pil_resized = pil_img.resize((new_w, target_h), Image.LANCZOS)

        out = np.asarray(pil_resized)

        # Convert back to float if that’s what you started with
        if original_dtype != np.uint8:
            out = out.astype(np.float32) / 255.0

        return out

    shap_arr = resize_keep_aspect(shap_arr, target_h)
    part_arr = resize_keep_aspect(part_arr, target_h)

    # 3) compute widths for a neat gridspec
    w1, w2 = shap_arr.shape[1], part_arr.shape[1]
    dpi    = 150
    fig_w, fig_h = (w1 + w2) / dpi, target_h / dpi

    fig, (ax1, ax2) = plt.subplots(
        1, 2,
        figsize=(fig_w, fig_h),
        dpi=dpi,
        gridspec_kw={'width_ratios': [w1, w2]},
        facecolor='white'
    )
    for ax, img in ((ax1, shap_arr), (ax2, part_arr)):
        ax.imshow(img)
        ax.axis('off')

    plt.tight_layout(pad=0)

    pdf_path = os.path.join(output_dir, os.path.splitext(img_name)[0] + f"_{FONT_SIZE}.pdf")
    fig.savefig(pdf_path, bbox_inches='tight', facecolor='white', format='pdf')
#     plt.show()
    plt.close(fig)

    print(f"✅ Saved combined PDF: {pdf_path}")

#################

    # ลบภาพเดี่ยว (ถ้าไม่ต้องการเก็บไว้)
    os.remove(shap_img_path)
#     break


['/data4-4tb/kgreathoo/plots_result_akara/38160661_peak_idx_1_type_test3_TP.png']
['$\\omega$$^{(6)}$', 'Std. ($\\omega$)$^{(9)}$', 'Mean ($\\omega$)$^{(6)}$', 'Max ($\\omega$)$^{(9)}$', 'Max ($\\omega$)$^{(7)}$', 'Min ($\\omega$)$^{(6)}$', 'Mean ($\\omega$)$^{(9)}$', 'Min ($\\omega$)$^{(5)}$', 'Max ($\\omega$)$^{(8)}$', '$\\omega$$^{(5)}$']


Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


['/data4-4tb/kgreathoo/plots_result_akara/38160661_peak_idx_1_type_test3_TP.png']
✅ Image: /data4-4tb/kgreathoo/plots_result_akara/38160661_peak_idx_1_type_test3_TP.png
🖼️ Title: 38160661 Peak Index 1 Type Test3 Tp
✅ Saved combined PDF: shap_outputs_1_10_new_great/38160661_peak_idx_1_type_test3_TP_96.pdf
['/data4-4tb/kgreathoo/plots_result_akara/54380859_peak_idx_2_type_test1_FN.png']
['$\\omega$$^{(6)}$', 'Std. ($\\omega$)$^{(9)}$', 'Mean ($\\omega$)$^{(6)}$', 'Max ($\\omega$)$^{(9)}$', 'Max ($\\omega$)$^{(7)}$', 'Min ($\\omega$)$^{(6)}$', 'Mean ($\\omega$)$^{(9)}$', 'Min ($\\omega$)$^{(5)}$', 'Max ($\\omega$)$^{(8)}$', '$\\omega$$^{(5)}$']


Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


['/data4-4tb/kgreathoo/plots_result_akara/54380859_peak_idx_2_type_test1_FN.png']
✅ Image: /data4-4tb/kgreathoo/plots_result_akara/54380859_peak_idx_2_type_test1_FN.png
🖼️ Title: 54380859 Peak Index 2 Type Test1 Fn
✅ Saved combined PDF: shap_outputs_1_10_new_great/54380859_peak_idx_2_type_test1_FN_96.pdf
['/data4-4tb/kgreathoo/plots_result_akara/54354537_peak_idx_1_type_test0_TP.png']
['$\\omega$$^{(6)}$', 'Std. ($\\omega$)$^{(9)}$', 'Mean ($\\omega$)$^{(6)}$', 'Max ($\\omega$)$^{(9)}$', 'Max ($\\omega$)$^{(7)}$', 'Min ($\\omega$)$^{(6)}$', 'Mean ($\\omega$)$^{(9)}$', 'Min ($\\omega$)$^{(5)}$', 'Max ($\\omega$)$^{(8)}$', '$\\omega$$^{(5)}$']


...

['/data4-4tb/kgreathoo/plots_result_akara/53798105_peak_idx_10_type_test2_TN.png']
✅ Image: /data4-4tb/kgreathoo/plots_result_akara/53798105_peak_idx_10_type_test2_TN.png
🖼️ Title: 53798105 Peak Index 10 Type Test2 Tn
✅ Saved combined PDF: shap_outputs_1_10_new_great/53798105_peak_idx_10_type_test2_TN_96.pdf
['/data4-4tb/kgreathoo/plots_result_akara/53882768_peak_idx_4_type_test3_TN.png']
['$\\omega$$^{(6)}$', 'Std. ($\\omega$)$^{(9)}$', 'Mean ($\\omega$)$^{(6)}$', 'Max ($\\omega$)$^{(9)}$', 'Max ($\\omega$)$^{(7)}$', 'Min ($\\omega$)$^{(6)}$', 'Mean ($\\omega$)$^{(9)}$', 'Min ($\\omega$)$^{(5)}$', 'Max ($\\omega$)$^{(8)}$', '$\\omega$$^{(5)}$']


Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


['/data4-4tb/kgreathoo/plots_result_akara/53882768_peak_idx_4_type_test3_TN.png']
✅ Image: /data4-4tb/kgreathoo/plots_result_akara/53882768_peak_idx_4_type_test3_TN.png
🖼️ Title: 53882768 Peak Index 4 Type Test3 Tn
✅ Saved combined PDF: shap_outputs_1_10_new_great/53882768_peak_idx_4_type_test3_TN_96.pdf
['/data4-4tb/kgreathoo/plots_result_akara/54203499_peak_idx_1_type_test2_TN.png']
['$\\omega$$^{(6)}$', 'Std. ($\\omega$)$^{(9)}$', 'Mean ($\\omega$)$^{(6)}$', 'Max ($\\omega$)$^{(9)}$', 'Max ($\\omega$)$^{(7)}$', 'Min ($\\omega$)$^{(6)}$', 'Mean ($\\omega$)$^{(9)}$', 'Min ($\\omega$)$^{(5)}$', 'Max ($\\omega$)$^{(8)}$', '$\\omega$$^{(5)}$']


Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


['/data4-4tb/kgreathoo/plots_result_akara/54203499_peak_idx_1_type_test2_TN.png']
✅ Image: /data4-4tb/kgreathoo/plots_result_akara/54203499_peak_idx_1_type_test2_TN.png
🖼️ Title: 54203499 Peak Index 1 Type Test2 Tn
✅ Saved combined PDF: shap_outputs_1_10_new_great/54203499_peak_idx_1_type_test2_TN_96.pdf


# compare with doctor

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

pred_df['HN'] = pred_df['HN'].astype(str)
doctor = pd.read_csv('Pre_Analyze_NPH_21012023.csv')

    
def predict_nph(score):
    if 3 <= score <= 4:
        return -1
    return score/12 >=0.5

pred_df = pred_df
# Apply the function to the 'Total score' column to create a new 'Predicted NPH' column
doctor['Predicted NPH'] = doctor['Total score (12)'].apply(predict_nph)
doctor['HN']=doctor['HN'].astype(int).astype(str)
pred_df['HN']=pred_df['HN'].str[0:8]
merged_df = pd.merge(doctor, pred_df, on='HN', how='left')
merged_df= merged_df[['HN','Predicted NPH','y_pred_model','y_prob','Clinical (NPH 1 normal 0)','proba_score']]
# merged_df= merged_df[['HN','Predicted NPH','y_pred','Clinical (NPH 1 normal 0)']]
filtered_doctor = merged_df[merged_df['Predicted NPH'] != -1]
model = pd.DataFrame()
doctor_score= pd.DataFrame()
model[['y','y_prob','y_pred','HN']]=filtered_doctor[['Clinical (NPH 1 normal 0)','y_prob','y_pred_model','HN']]
doctor_score[['y','y_pred']]=filtered_doctor[['Clinical (NPH 1 normal 0)','Predicted NPH']]
model=model.dropna()
# Drop rows with null values in 'y' or 'y_pred' columns
doctor_score.dropna(subset=['y', 'y_pred'], inplace=True)

# Ensure that 'y_pred' is an integer
model['y_pred'] = model['y_pred'].astype('int')
y_true = model['y']
y_pred = model['y_pred']
y_prob = np.vstack(model['y_prob'].values) 
confusion_mat = confusion_matrix(y_true, y_pred, labels=[0, 1])
logit_roc_auc = roc_auc_score(y_true, y_prob[:, 1])
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred), f' (area = {logit_roc_auc:.2f})')
print('cohen',cohen_kappa_score(y_true, y_pred))

# model = pd.DataFrame()
# doctor_score= pd.DataFrame()
# model[['y','y_pred','HN']]=filtered_doctor[['Clinical (NPH 1 normal 0)','y_pred_model','HN']]
# doctor_score[['y','y_pred']]=filtered_doctor[['Clinical (NPH 1 normal 0)','Predicted NPH']]
# model=model.dropna()
# # Drop rows with null values in 'y' or 'y_pred' columns
# doctor_score.dropna(subset=['y', 'y_pred'], inplace=True)
TP, FN = cm[0]
FP, TN = cm[1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
f1_score = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
roc_auc = roc_auc_score(y_true, y_prob[:, 1])
cohen_kappa = cohen_kappa_score(y_true, y_pred)

# print("Acc\tSensiy\tSpec\tF1-Score\tTP\tTN\tFP\tFN\tAU-ROC\tCohen")
print(f"{accuracy:.3f}\t{sensitivity:.3f}\t{specificity:.3f}\t{f1_score:.3f}\t"
      f"{TP}\t{TN}\t{FP}\t{FN}\t{roc_auc:.3f}\t{cohen_kappa:.3f}")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

doctor = pd.read_csv('/media/akara_bkp/nph_kgreat/kgreathoo/csv/Pre_Analyze_NPH_21012023.csv')

    
def predict_nph(score):
    if 3 <= score <= 4:
        return -1
    return score/12 >=0.5

pred_df = pred_df
# Apply the function to the 'Total score' column to create a new 'Predicted NPH' column
doctor['Predicted NPH'] = doctor['Total score (12)'].apply(predict_nph)
doctor['HN']=doctor['HN'].astype(int).astype(str)
pred_df['HN']=pred_df['HN'].str[0:8]
merged_df = pd.merge(doctor, pred_df, on='HN', how='left')
merged_df= merged_df[['HN','Predicted NPH','y_pred_model','y_prob','Clinical (NPH 1 normal 0)','proba_score']]
# merged_df= merged_df[['HN','Predicted NPH','y_pred','Clinical (NPH 1 normal 0)']]
filtered_doctor = merged_df[merged_df['Predicted NPH'] == -1]
model = pd.DataFrame()
doctor_score= pd.DataFrame()
model[['y','y_prob','y_pred','HN']]=filtered_doctor[['Clinical (NPH 1 normal 0)','y_prob','y_pred_model','HN']]
doctor_score[['y','y_pred']]=filtered_doctor[['Clinical (NPH 1 normal 0)','Predicted NPH']]
model=model.dropna()
# Drop rows with null values in 'y' or 'y_pred' columns
doctor_score.dropna(subset=['y', 'y_pred'], inplace=True)

# Ensure that 'y_pred' is an integer
model['y_pred'] = model['y_pred'].astype('int')
y_true = model['y']
y_pred = model['y_pred']
y_prob = np.vstack(model['y_prob'].values) 
confusion_mat = confusion_matrix(y_true, y_pred, labels=[0, 1])
logit_roc_auc = roc_auc_score(y_true, y_prob[:, 1])
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred), f' (area = {logit_roc_auc:.2f})')
print('cohen',cohen_kappa_score(y_true, y_pred))

model = pd.DataFrame()
doctor_score= pd.DataFrame()
model[['y','y_pred','HN']]=filtered_doctor[['Clinical (NPH 1 normal 0)','y_pred_model','HN']]
doctor_score[['y','y_pred']]=filtered_doctor[['Clinical (NPH 1 normal 0)','Predicted NPH']]
model=model.dropna()
# Drop rows with null values in 'y' or 'y_pred' columns
doctor_score.dropna(subset=['y', 'y_pred'], inplace=True)
TP, FN = cm[0]
FP, TN = cm[1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
f1_score = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
roc_auc = roc_auc_score(y_true, y_prob[:, 1])
cohen_kappa = cohen_kappa_score(y_true, y_pred)

# print("Acc\tSensiy\tSpec\tF1-Score\tTP\tTN\tFP\tFN\tAU-ROC\tCohen")
print(f"{accuracy:.3f}\t{sensitivity:.3f}\t{specificity:.3f}\t{f1_score:.3f}\t"
      f"{TP}\t{TN}\t{FP}\t{FN}\t{roc_auc:.3f}\t{cohen_kappa:.3f}")

In [ ]:
# merged_df.to_csv(f'{MODEL_TYPE}_prediction_18042024.csv')

In [ ]:
# MODEL_TYPE

In [ ]:
# merged_df